import libraries and read in data

In [0]:
import pandas as pd
import numpy as np
postal = pd.read_excel('/content/Cap_W3.xlsx')

remove unassigned boroughs and replace unassigned neighborhoods with borough name

In [0]:
postal = postal.loc[postal['Borough'] != 'Not assigned']
 
postal = postal.replace(to_replace ='Not assigned', value = postal['Neighborhood'])


In [149]:
postal.PostalCode.unique()
postal.head()

,PostalCode,Borough,Neighborhood
3,M9W,Etobicoke,Northwest
4,M9V,Etobicoke,Albion Gardens
5,M9V,Etobicoke,Beaumond Heights
6,M9V,Etobicoke,Humbergate
7,M9V,Etobicoke,Jamestown


group data by PostalCode, joining the Neighbors values with ","

In [150]:
grouped = postal.groupby('PostalCode')['Neighborhood'].apply(', '.join)
grouped = grouped.to_frame()
type(grouped)
print(grouped)

                                                 Neighborhood
PostalCode                                                   
M1B                                            Rouge, Malvern
M1C                    Highland Creek, Rouge Hill, Port Union
M1E                         Guildwood, Morningside, West Hill
M1G                                                    Woburn
M1H                                                 Cedarbrae
...                                                       ...
M9N                                                    Weston
M9P                                                 Westmount
M9R         Kingsview Village, Martin Grove Gardens, Richv...
M9V         Albion Gardens, Beaumond Heights, Humbergate, ...
M9W                                                 Northwest

[103 rows x 1 columns]


merge datasets to add back 'Borough' column

In [163]:
import pandas as pd
df = pd.merge(grouped, postal2, on='PostalCode', how='inner')
df = df[['PostalCode', 'Borough', 'Neighborhood']]
print(df)

    PostalCode      Borough                                       Neighborhood
0          M1B  Scarborough                                     Rouge, Malvern
1          M1B  Scarborough                                     Rouge, Malvern
2          M1C  Scarborough             Highland Creek, Rouge Hill, Port Union
3          M1C  Scarborough             Highland Creek, Rouge Hill, Port Union
4          M1C  Scarborough             Highland Creek, Rouge Hill, Port Union
..         ...          ...                                                ...
205        M9V    Etobicoke  Albion Gardens, Beaumond Heights, Humbergate, ...
206        M9V    Etobicoke  Albion Gardens, Beaumond Heights, Humbergate, ...
207        M9V    Etobicoke  Albion Gardens, Beaumond Heights, Humbergate, ...
208        M9V    Etobicoke  Albion Gardens, Beaumond Heights, Humbergate, ...
209        M9W    Etobicoke                                          Northwest

[210 rows x 3 columns]


drop merged duplicates and print shape

In [171]:
df = df.drop_duplicates()
df.head(12)
df.shape

(103, 3)

add postal code coordinates

In [181]:
geo = pd.read_csv('/content/Geospatial_Coordinates.csv')
geo.columns = ['PostalCode', 'Latitude', 'Longitude']
geo.columns

geo = pd.merge(df, geo, on='PostalCode', how='inner')
geo.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


import necessary libraries and dependancies for mapping data

In [0]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium 
import folium # map rendering library

print('Libraries imported.')

Select Toronto Boroughs

In [192]:
geo2 = geo[geo['Borough'].str.contains("Toronto")]
geo2['Borough'].values
geo2.columns

Index(['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

make folium map with neighborhood markers

In [198]:
latitude = geo['Latitude']
longitude = geo['Longitude']

map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(geo['Latitude'], geo['Longitude'], geo['Borough'], geo['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor